# **Retrival Augmented Generation (RAG) using Falcon 7B**

---
<a target="_blank" href="https://colab.research.google.com/github/samsonthomas951/Large-Language-Model-Notebooks/blob/main/2.Retrival%20Arguemented%20Generation%20with LLMs/Falcon_7b_RAG-lanchain.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg"></a>

**RAG** - Thhis is an AI framework for retrieving facts from an external knowledge basr to group large languade models(LLMs) on the most accurate, up-to-date infornation and to give users insight into LLMs' generative process

In this note book I will not download the Falcon 7b model rather I will use the hosted falcon 7b model on huggingface.

You will be required to have huggingface access token [here](https://huggingface.co/settings/tokens), after that you'll be ready to run this note book. 

I chose a model that is hosted from huugingface because they have the computation resources to run the Falcon 7B model fast enough for RAG.

You will be required to provide any PDF documents that you'll want to ask questions from.


Installing the required libraries

* faiss-cpu - (Facebook AI similarity search) allows developers to quickly search embeddings of multimedia documents that are similar to each other.

* pypdf - used to readthe pdf documentd

* sentence_transformers - used to capture contextual relationships in sequential data

* langchain - this uses the power of the model combined with data sources to create applications that are contextr aware: connect llms to sources of context(promping intructions, few shot examples, content to ground its information)


In [12]:
!pip install faiss-cpu pypdf sentence_transformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 3.0 MB/s eta 0:00:00


Importing the indtalled libraries

In [33]:

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

Load text and split into chunks:

## How to load your pdf to google colab.
1. Drag n Drop
 - select the pdf file that you want to ask questions about
 - drag it and drop it on the content folder on colab

2. Uploading via code
 - you can add a cell below and write these code
    ```py
        
        from google.colab import files
        #uploading a single file
        uploaded = files.upload()
    ```
 - using a document that is in your google drive
    ```py
        from google.colab import drive
        drive.mount('/content/drive')
    ```


In [34]:
#enter the path to your document
loader = PyPDFLoader("/content/Topic 6 Notes.pdf")
documents = loader.load()

# Split the documents into chunks
splitter = RecursiveCharacterTextSplitter()
chunks = splitter.split_documents(documents)

# Convert the chunks to strings
chunk_texts = [chunk.page_content for chunk in chunks]


In [36]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# using the huggingfaceEmbeddings
embeddings = HuggingFaceEmbeddings()

Embed the chunks and store the embeddings in a vector database:

In [37]:
from langchain.vectorstores import FAISS
vectorStore = FAISS.from_texts(chunk_texts, embeddings)

In [42]:
# from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_HUGGINGFACE_ACCESS_TOKEN_HERE" #get one at https://huggingface.co/settings/tokens

llm=HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"temperature":0.1 ,"max_length":1024})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


Create a question answering chain:

In [43]:
from langchain.chains import RetrievalQA
from langchain.schema import retriever

chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorStore.as_retriever())


Ask a question:

In [44]:
query="What is the Requirement of a Data Dictionary?"
chain.run(query)

' The requirement of a data dictionary is to provide a centralized repository of information about the data. It is used to store and retrieve data from the database. It is also used to validate the data flow and ensure that the data is consistent and accurate.'